# Classification and Clustering

In [13]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA

### 1. Create training and evaluation sets

In [ ]:
# load the data set 
# extract the features 
# tokens with TFID normalisation on the raw counts  - run the tokeniser and then add to the dataframe 
# NER with TFID normalisation on the raw counts 



In [ ]:
# write to svmlight 
sklearn.datasets.dump_svmlight_file()

### 2. Multilabel Classification 

In [14]:
X, Y = make_multilabel_classification(n_classes=5, n_labels=5,
                                      allow_unlabeled=True,
                                      random_state=1)

In [15]:
X[0]

array([5., 3., 2., 5., 7., 1., 1., 2., 0., 2., 0., 3., 9., 1., 1., 2., 4.,
       3., 0., 4.])

In [16]:
# we need to get the labels encoded like this for multilabel classification
# step 1, find out how many unique labels we have in the corpus
# have a look at the distribution of the label. are there significant outliers. if few, consider removing them
# else, we may consider augmenting datapoints with SMOTE 
Y[0]

array([1, 1, 0, 0, 1])

### 3. Clustering with Graph propagation extension

In [1]:
# i think we can try this. since our data set is not very big. we could use our initial clusters as a feature and retrain 
# the classifier 

array([5., 3., 2., 5., 7., 1., 1., 2., 0., 2., 0., 3., 9., 1., 1., 2., 4.,
       3., 0., 4.])

array([1, 1, 0, 0, 1])